In [1]:
"""
Welcome to the S2AND tutorial!

We will cover a few aspects of the S2AND pipeline:

(1) Load a test dataset.
(2) Fit a pairwise model + bells & whistles.
(3) Fit a clusterer.
(4) Evaluate the pairwise model and the clusterer.
""";

In [2]:
import os

os.environ["OMP_NUM_THREADS"] = "8"

import json
import copy
import argparse
import logging
import pickle
from typing import Dict, Any, Optional, List
from collections import defaultdict

import numpy as np
import pandas as pd

from s2and.data import PDData
from s2and.featurizer import featurize, FeaturizationInfo
from s2and.model import PairwiseModeler, Clusterer, FastCluster
from s2and.eval import pairwise_eval, cluster_eval, facet_eval
from s2and.consts import FEATURIZER_VERSION, DEFAULT_CHUNK_SIZE, PROJECT_ROOT_PATH
from s2and.file_cache import cached_path
from s2and.plotting_utils import plot_facets
from hyperopt import hp

2022-10-11 00:08:46,283 - s2and - WARNING - You haven't set `main_data_dir` in data/path_config.json! Using data/ as default data directory.


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [3]:
# this is the random seed we used for the ablations table
random_seed = 42
# number of cpus to use
n_jobs = 4

In [4]:
# we're going to load the arnetminer dataset
# and assume that you have it already downloaded to the `S2AND/data/` directory

dataset_name = 'test'
DATA_DIR = os.path.join(PROJECT_ROOT_PATH, 'data')

pddata = PDData(
    papers=os.path.join(DATA_DIR, dataset_name + "_papers.json"),
    name=dataset_name,
    mode="train",  # can also be 'inference' if just predicting
    clusters=os.path.join(DATA_DIR, dataset_name + "_clusters.json"),
    train_pairs=None,  # in case you have predefined splits for the pairwise models
    val_pairs=None,
    test_pairs=None,
    train_pairs_size=100000,  # how many training pairs for the pairwise models?
    val_pairs_size=10000,
    test_pairs_size=10000,
    n_jobs=n_jobs,
    load_name_counts=False,
    random_seed=random_seed,
)

2022-10-11 00:08:49,084 - s2and - INFO - loading papers
2022-10-11 00:08:49,325 - s2and - INFO - loaded papers
2022-10-11 00:08:49,326 - s2and - INFO - loading clusters
2022-10-11 00:08:49,328 - s2and - INFO - loaded clusters, loading specter
2022-10-11 00:08:49,328 - s2and - INFO - loaded specter, loading cluster seeds
2022-10-11 00:08:49,329 - s2and - INFO - loaded cluster seeds
2022-10-11 00:08:49,330 - s2and - INFO - making paper to cluster id
2022-10-11 00:08:49,332 - s2and - INFO - made paper to cluster id
2022-10-11 00:08:49,340 - s2and - INFO - preprocessing papers
Preprocessing papers: 100%|██████████| 3601/3601 [00:01<00:00, 1869.53it/s]
2022-10-11 00:08:51,412 - s2and - INFO - preprocessed papers


In [5]:
# to train the pairwise model, we define which feature categories to use
# here it is all of them
features_to_use = [
    "author_similarity",
    "venue_similarity",
    "year_diff",
    "title_similarity",
    "abstract_similarity",
    "paper_quality",
]

# we store all the information about the features in this convenient wrapper
featurization_info = FeaturizationInfo(features_to_use=features_to_use, featurizer_version=FEATURIZER_VERSION)

# now we can actually go and get the pairwise training, val and test data
train, val, test = featurize(pddata, featurization_info, n_jobs=4, use_cache=False, chunk_size=DEFAULT_CHUNK_SIZE, nan_value=np.nan)  # type: ignore
X_train, y_train, nameless_X_train = train
X_val, y_val, nameless_X_val = val
X_test, y_test, nameless_X_test = test

2022-10-11 00:08:53,543 - s2and - INFO - featurizing train
2022-10-11 00:08:53,545 - s2and - INFO - Creating 2107 pieces of work
2022-10-11 00:08:53,562 - s2and - INFO - Created pieces of work
2022-10-11 00:08:53,563 - s2and - INFO - Cached changed, doing 2107 work in parallel
2022-10-11 00:08:53,847 - s2and - INFO - Work completed
2022-10-11 00:08:53,850 - s2and - INFO - Making numpy arrays for features and labels
2022-10-11 00:08:53,852 - s2and - INFO - Numpy arrays made
2022-10-11 00:08:53,855 - s2and - INFO - featurized train, featurizing val
2022-10-11 00:08:53,856 - s2and - INFO - Creating 418 pieces of work
2022-10-11 00:08:53,859 - s2and - INFO - Created pieces of work
2022-10-11 00:08:53,860 - s2and - INFO - Cached changed, doing 418 work in parallel
2022-10-11 00:08:53,984 - s2and - INFO - Work completed
2022-10-11 00:08:53,986 - s2and - INFO - Making numpy arrays for features and labels
2022-10-11 00:08:53,989 - s2and - INFO - Numpy arrays made
2022-10-11 00:08:53,990 - s2an

In [6]:
# now we define and fit the pairwise modelers
pairwise_modeler = PairwiseModeler(
    n_iter=25,  # number of hyperparameter search iterations
    estimator=None,  # this will use the default LightGBM classifier
    search_space=None,  # this will use the default LightGBM search space
    monotone_constraints=featurization_info.lightgbm_monotone_constraints,  # we use monotonicity constraints to make the model more sensible
    random_state=random_seed,
)
pairwise_modeler.fit(X_train, y_train, X_val, y_val)

100%|██████████| 25/25 [00:05<00:00,  4.78trial/s, best loss: -0.9976514560972196]


In [7]:
# now we can fit the clusterer itself
clusterer = Clusterer(
    featurization_info,
    pairwise_modeler.classifier,  # the actual pairwise classifier
    cluster_model=FastCluster(linkage='average'),  # average linkage agglomerative clustering
    search_space={"eps": hp.uniform("choice", 0, 1)},  # the hyperparemetrs for the clustering algorithm
    n_jobs=n_jobs,
    use_cache=False,
    random_state=random_seed,
)
clusterer.fit(pddata)
print(clusterer.best_params)

2022-10-11 00:09:01,463 - s2and - INFO - Fitting clusterer
2022-10-11 00:09:01,536 - s2and - INFO - Making 10 distance matrices
2022-10-11 00:09:01,537 - s2and - INFO - Initializing pairwise_probas
2022-10-11 00:09:01,538 - s2and - INFO - Pairwise probas initialized, starting making all pairs
2022-10-11 00:09:01,539 - s2and - INFO - Featurizing batch 0/1
2022-10-11 00:09:01,539 - s2and - INFO - Getting constraints
2022-10-11 00:09:01,582 - s2and - INFO - Creating 16719 pieces of work
2022-10-11 00:09:01,605 - s2and - INFO - Created pieces of work
2022-10-11 00:09:01,606 - s2and - INFO - Cached changed, doing 16719 work in parallel


Finished loading model, total used 770 iterations


Doing work: 100%|██████████| 16719/16719 [00:00<00:00, 31032.70it/s]
2022-10-11 00:09:02,297 - s2and - INFO - Work completed
2022-10-11 00:09:02,299 - s2and - INFO - Making numpy arrays for features and labels
2022-10-11 00:09:02,304 - s2and - INFO - Numpy arrays made
2022-10-11 00:09:02,309 - s2and - INFO - Making predict flags
2022-10-11 00:09:02,315 - s2and - INFO - Pairwise classification
2022-10-11 00:09:02,355 - s2and - INFO - Starting to make matrices
Writing matrices: 100%|██████████| 16719/16719 [00:00<00:00, 1011519.03it/s]
2022-10-11 00:09:02,379 - s2and - INFO - 10 distance matrices made


100%|██████████| 25/25 [00:00<00:00, 189.33trial/s, best loss: -0.992]

2022-10-11 00:09:02,520 - s2and - INFO - Clusterer fit



{'eps': 0.6193434304652304}


In [8]:
# but how good are our models? 
# first, let's look at the quality of the pairwise evaluation
pairwise_metrics = pairwise_eval(
    X_test,
    y_test,
    pairwise_modeler,
    os.path.join(PROJECT_ROOT_PATH, "data", "tutorial_figures"),  # where to put the figures
    "tutorial_figures",  # what to call the figures
    featurization_info.get_feature_names(),
    skip_shap=False,  # if your model isn't a tree-based model, you should put True here and it will not make SHAP figures
)
print(pairwise_metrics)

C extension was not built during install!


LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
All-NaN slice encountered


{'AUROC': 0.994, 'Average Precision': 0.993, 'F1': 0.983, 'Precision': 0.985, 'Recall': 0.982}


In [9]:
# we can do the same thing for the clustering performance
cluster_metrics, b3_metrics_per_signature = cluster_eval(
    pddata,
    clusterer,
    split="test",  # which part of the data to evaluate on, can also be 'val'
    use_s2_clusters=False,  # set to true if you want to see how the old S2 system does
)
print(cluster_metrics)

2022-10-11 00:09:08,385 - s2and - INFO - Making 11 distance matrices
2022-10-11 00:09:08,386 - s2and - INFO - Initializing pairwise_probas
2022-10-11 00:09:08,387 - s2and - INFO - Pairwise probas initialized, starting making all pairs
2022-10-11 00:09:08,388 - s2and - INFO - Featurizing batch 0/1
2022-10-11 00:09:08,389 - s2and - INFO - Getting constraints
2022-10-11 00:09:08,410 - s2and - INFO - Creating 6717 pieces of work
2022-10-11 00:09:08,419 - s2and - INFO - Created pieces of work
2022-10-11 00:09:08,420 - s2and - INFO - Cached changed, doing 6717 work in parallel
2022-10-11 00:09:08,806 - s2and - INFO - Work completed
2022-10-11 00:09:08,809 - s2and - INFO - Making numpy arrays for features and labels
2022-10-11 00:09:08,812 - s2and - INFO - Numpy arrays made
2022-10-11 00:09:08,816 - s2and - INFO - Making predict flags
2022-10-11 00:09:08,819 - s2and - INFO - Pairwise classification
2022-10-11 00:09:08,846 - s2and - INFO - Starting to make matrices
Writing matrices: 100%|█████

{'B3 (P, R, F1)': (1.0, 0.981, 0.99), 'Pred bigger ratio (mean, count)': (1.0, 75), 'True bigger ratio (mean, count)': (2.0, 4)}
